# ダブルクロスバリデーションの確認（３分割時）

scikit-learn のライブラリーを使用した、ダブルクロスバリデーションの手法についての確認です。

こちらの記述を参考にさせていただきました。

http://univprof.com/archives/16-06-12-3889388.html

## (1) テストデータ／環境準備

マイオペで使用しているテストデータ（learning/tests/engine/fixtures/ 配下のCSVファイル）をベースに動作確認を行います。

動作確認にあたっては、MySQLdb に接続できないため、ローカル環境テスト用の Bot クラスを使用しています。

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定した複数件のファイルを使用します。
'''
csv_file_names = [
    'test_daikin_conversation.csv',
]
copied_csv_file_paths = TestTool.copy_testdata_csv(learning_dir, csv_file_names)
copied_csv_file_paths

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv]


['/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv']

## (2) TF-IDFベクターの準備

Bot クラス内に組み込まれている __build_training_set_from_csv 関数をバラして実行しています。

In [3]:
'''
    初期設定
    データファイル、エンコードを指定
    内容は、learn.py を参考にしました。    
'''
from learning.core.learn.learning_parameter import LearningParameter
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': None,
    # 'algorithm': LearningParameter.ALGORITHM_NAIVE_BAYES
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    # 'params_for_algorithm': { 'C': 200 }
    'params_for_algorithm': {}
}
learning_parameter = LearningParameter(attr)

bot_id = 7777
csv_file_encoding = 'utf-8'

### (2-1) 訓練データのTF-IDFベクター

In [4]:
'''
    訓練データの生成（内部で TF-IDF 処理を実行）
    
    text_array.py における TF-IDF 処理では、
    以下の通り「する」をストップワード化指定しております
    
    class TextArray:
        :
        def __build_vectorizer(self):
            :
            vectorizer = TfidfVectorizer(use_idf=False, token_pattern=u'(?u)\\b\\w+\\b', stop_words=['する'])
'''
#from learning.core.training_set.training_message_from_csv import TrainingMessageFromCsv
from prototype.modules.training_message_from_csv import TrainingMessageFromCsv
training_set = TrainingMessageFromCsv(bot_id, copied_csv_file_paths, learning_parameter, encoding=csv_file_encoding)
build_training_set_from_csv = training_set.build()

X = build_training_set_from_csv.x
y = build_training_set_from_csv.y

2017/03/28 PM 08:30:16 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 17443
2017/03/28 PM 08:30:16 TextArray#__init__ start
2017/03/28 PM 08:30:25 TextArray#to_vec start
2017/03/28 PM 08:30:25 TextArray#to_vec end


In [5]:
n_sample = X.shape[0]
n_feature = X.shape[1]
print("sample=%d, feature=%d" % (n_sample, n_feature))

sample=17443, feature=1967


## (3) ダブルクロスバリデーションを使用しない場合

### (3-1) モデル選択フェーズ

C = 200 と決定されました。

In [6]:
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
import numpy as np

''' 
    マイオペのプロダクション・コードから、
    該当部分を抽出して実行しました
    
    ロジスティック回帰モデルを
    ４通りのハイパーパラメータでグリッドサーチ
'''
params = {'C': [10, 100, 140, 200]}
grid = GridSearchCV(LogisticRegression(), param_grid=params)
grid.fit(X, y)
estimator = grid.best_estimator_
grid.best_params_

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 2 members, which is too few. The minimum num

{'C': 200}

### (3-2) モデル評価フェーズ

Accuracy = 0.986 と評価されました。

In [7]:
from sklearn.cross_validation import ShuffleSplit
from sklearn.cross_validation import cross_val_score

def __accuracy_score(estimator, X, y):
    y_pred = estimator.predict(X)
    probabilities = estimator.predict_proba(X)
    max_probabilities = np.max(probabilities, axis=1)

    # 予測結果と実際を比較
    bools = y == y_pred
    # しきい値を超えているもののみをTrueにする
    bools = bools == (max_probabilities > 0.5)
    # 正当率を算出
    score = np.sum(bools) / np.size(bools, axis=0)
    return score

''' 
    マイオペのプロダクション・コードから、
    該当部分を抽出して実行しました
    
    反復＝1回、テストデータ比率＝0.25
'''
cv = ShuffleSplit(X.shape[0], n_iter=1, test_size=0.25, random_state=0)
accuracy = np.mean(cross_val_score(estimator, X, y, cv=cv, scoring=__accuracy_score))

In [8]:
accuracy

0.98601238248108236

## (4) ダブルクロスバリデーションを使用する場合

こちらの記述における手順を再現しています。

http://univprof.com/archives/16-06-12-3889388.html

ただし、データ分割は A, B, C の３分割としています。（上記例では A, B の２分割）

### (4-0) データを分割

元の訓練データを分割し、各サブデータセット（A, B, C）を作成します。

データを重複無しに３分割する方法が見つからなかったので、ShuffleSplit を２段で実行しています。

（ShuffleSplit を１段で３回繰り返すと、３分割されたデータ間で重複が発生してしまうことが判明したため、これを回避します）

#### 訓練データを２：１で分け、１の部分をサブデータセット A とします。

In [9]:
y = np.array(build_training_set_from_csv.y)

In [10]:
'''
    分割されて取得されたテストデータ（３：１でShuffleSplitした１の部分）を
    サブデータセット A と命名

'''
cv_1 = ShuffleSplit(X.shape[0], n_iter=1, test_size=0.333333, random_state=0)

iter = 1
for train_index, test_index in cv_1: # n_iter=1 なのでこのループは１回だけしか回りません。
    subdataset_T_X, subdataset_A_X = X[train_index], X[test_index]
    subdataset_T_y, subdataset_A_y = y[train_index], y[test_index]
    print('iteration=%d' % iter)
    iter += 1

iteration=1


In [11]:
subdataset_A_X

<5815x1967 sparse matrix of type '<class 'numpy.float64'>'
	with 50019 stored elements in Compressed Sparse Row format>

In [12]:
len(subdataset_A_y)

5815

#### さらに、残りの訓練データを１：１で分け、それぞれサブデータセット B, C とします。

In [13]:
cv_2 = ShuffleSplit(subdataset_T_X.shape[0], n_iter=1, test_size=0.5, random_state=0)

iter = 1
for train_index, test_index in cv_2: # n_iter=1 なのでこのループは１回だけしか回りません。
    subdataset_B_X, subdataset_C_X = subdataset_T_X[train_index], subdataset_T_X[test_index]
    subdataset_B_y, subdataset_C_y = subdataset_T_y[train_index], subdataset_T_y[test_index]
    print('iteration=%d' % iter)
    iter += 1

iteration=1


In [14]:
subdataset_B_X

<5814x1967 sparse matrix of type '<class 'numpy.float64'>'
	with 49732 stored elements in Compressed Sparse Row format>

In [15]:
len(subdataset_B_y)

5814

In [16]:
subdataset_C_X

<5814x1967 sparse matrix of type '<class 'numpy.float64'>'
	with 49383 stored elements in Compressed Sparse Row format>

In [17]:
len(subdataset_C_y)

5814

In [18]:
'''
    サブデータセットごとのテストデータ件数を表示します。
    合計はサンプル全体の件数になります。
'''
print('Iteration data count: 1st=%d, 2nd=%d, 3rd=%d, total=%d' % (
    subdataset_A_X.shape[0], 
    subdataset_B_X.shape[0], 
    subdataset_C_X.shape[0],
    subdataset_A_X.shape[0] + subdataset_B_X.shape[0] + subdataset_C_X.shape[0]
))

Iteration data count: 1st=5815, 2nd=5814, 3rd=5814, total=17443


### (4-1) サブデータセット A を使った予測

#### サブデータセット B, C を使って学習します。

In [19]:
from scipy.sparse import vstack

X_train_1 = vstack((subdataset_B_X, subdataset_C_X))
X_train_1

<11628x1967 sparse matrix of type '<class 'numpy.float64'>'
	with 99115 stored elements in Compressed Sparse Row format>

In [20]:
y_train_1 = np.concatenate((subdataset_B_y, subdataset_C_y))
len(y_train_1)

11628

In [21]:
''' 
    サブデータセット B+C を使用し
    ロジスティック回帰モデルを使用し学習実行
'''
params = {'C': [10, 100, 140, 200]}
grid = GridSearchCV(LogisticRegression(), param_grid=params)
grid.fit(X_train_1, y_train_1)
estimator = grid.best_estimator_
grid.best_params_

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


{'C': 140}

#### サブデータセット A を使って予測します。

In [22]:
''' 
    サブデータセット A を使用し
    予測結果を取得
'''
y_pred_A = estimator.predict(subdataset_A_X)
probabilities = estimator.predict_proba(subdataset_A_X)
y_proba_A = np.max(probabilities, axis=1)

In [23]:
y_pred_A

array([3687, 3402, 3514, ..., 3963, 3813, 3704])

In [24]:
y_proba_A

array([ 0.97712632,  0.96553817,  0.97512191, ...,  0.98186175,
        0.96870209,  0.98267401])

In [25]:
''' 
    プロダクション・コードを拝借・・・
'''
def accuracy_score(y, y_pred, y_proba):
    # 予測結果と実際を比較
    bools = y == y_pred
    # しきい値を超えているもののみをTrueにする
    bools = bools == (y_proba > 0.5)
    # 正当率を算出
    score = np.sum(bools) / np.size(bools, axis=0)
    return score

In [26]:
''' 
    参考：サブデータセット A の正解率
'''
accuracy_A = accuracy_score(subdataset_A_y, y_pred_A, y_proba_A)
accuracy_A

0.98056749785038688

### (4-2) サブデータセット B を使った予測

#### サブデータセット A, C を使って学習します。

In [27]:
from scipy.sparse import vstack

X_train_2 = vstack((subdataset_A_X, subdataset_C_X))
X_train_2

<11629x1967 sparse matrix of type '<class 'numpy.float64'>'
	with 99402 stored elements in Compressed Sparse Row format>

In [28]:
y_train_2 = np.concatenate((subdataset_A_y, subdataset_C_y))
len(y_train_2)

11629

In [29]:
''' 
    サブデータセット A+C を使用し
    ロジスティック回帰モデルを使用し学習実行
'''
params = {'C': [10, 100, 140, 200]}
grid = GridSearchCV(LogisticRegression(), param_grid=params)
grid.fit(X_train_2, y_train_2)
estimator = grid.best_estimator_
grid.best_params_

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


{'C': 100}

#### サブデータセット B を使って予測します。

In [30]:
''' 
    サブデータセット B を使用し
    予測結果を取得
'''
y_pred_B = estimator.predict(subdataset_B_X)
probabilities = estimator.predict_proba(subdataset_B_X)
y_proba_B = np.max(probabilities, axis=1)

In [31]:
y_pred_B

array([3823, 3441, 3644, ..., 3961, 3807, 3757])

In [32]:
y_proba_B

array([ 0.97145497,  0.96865482,  0.94014989, ...,  0.97727819,
        0.96402621,  0.97244357])

In [33]:
''' 
    参考：サブデータセット B の正解率
'''
accuracy_B = accuracy_score(subdataset_B_y, y_pred_B, y_proba_B)
accuracy_B

0.97351221190230475

### (4-3) サブデータセット C を使った予測

#### サブデータセット A, B を使って学習します。

In [34]:
from scipy.sparse import vstack

X_train_3 = vstack((subdataset_A_X, subdataset_B_X))
X_train_3

<11629x1967 sparse matrix of type '<class 'numpy.float64'>'
	with 99751 stored elements in Compressed Sparse Row format>

In [35]:
y_train_3 = np.concatenate((subdataset_A_y, subdataset_B_y))
y_train_3

array([3687, 3402, 3514, ..., 3961, 3807, 3757])

In [36]:
''' 
    サブデータセット A+B を使用し
    ロジスティック回帰モデルを使用し学習実行
'''
params = {'C': [10, 100, 140, 200]}
grid = GridSearchCV(LogisticRegression(), param_grid=params)
grid.fit(X_train_3, y_train_3)
estimator = grid.best_estimator_
grid.best_params_

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


{'C': 140}

#### サブデータセット C を使って予測します。

In [37]:
''' 
    サブデータセット C を使用し
    予測結果を取得
'''
y_pred_C = estimator.predict(subdataset_C_X)
probabilities = estimator.predict_proba(subdataset_C_X)
y_proba_C = np.max(probabilities, axis=1)

In [38]:
y_pred_C

array([3402, 3959, 3529, ..., 3781, 3525, 3506])

In [39]:
y_proba_C

array([ 0.96611708,  0.97744889,  0.97377329, ...,  0.97662744,
        0.98495656,  0.96931191])

In [40]:
''' 
    参考：サブデータセット C の正解率
'''
accuracy_C = accuracy_score(subdataset_C_y, y_pred_C, y_proba_C)
accuracy_C

0.979188166494668

### (4-4) 全ての予測結果に対する正解率を算出

In [41]:
''' 
    予測結果をマージ
    ---> データセット中のすべてのサンプル（A, B, C）における予測値となります
'''
y_test_all = np.concatenate((subdataset_A_y, subdataset_B_y, subdataset_C_y))
y_pred_all = np.concatenate((y_pred_A, y_pred_B, y_pred_C))
y_proba_all = np.concatenate((y_proba_A, y_proba_B, y_proba_C))

In [42]:
y_pred_all

array([3687, 3402, 3514, ..., 3781, 3525, 3506])

In [43]:
y_proba_all

array([ 0.97712632,  0.96553817,  0.97512191, ...,  0.97662744,
        0.98495656,  0.96931191])

In [44]:
''' 
    全ての予測結果に対する正解率を算出
'''
accuracy_all = accuracy_score(y_test_all, y_pred_all, y_proba_all)
accuracy_all

0.97775611993349765